In [1]:
import matplotlib.pyplot as plt
import pandas 
import math
import numpy as np
import random
from numpy import linalg
from numpy import matrix
import time
%matplotlib inline

In [ ]:
alphas = np.arange(0.00, 0.50, 0.05)
#average_distance = []
performance = []
for alpha in alphas:
    start_time = time.time()
    data = open('data10')
    user_articles = [''] * 20
    article_features = {}
    payoff_for_user = [0.] * 20
    sum_reward_of_alg = 0.
    sum_reward_of_data = 0.
    distance = []
    z = {}
    matrix_A = {}
    matrix_A0 = np.matrix(np.identity(25))
    matrix_B = {}
    inversed_matrix_A = {}
    vector_b = {}
    vector_b0 = np.matrix(np.zeros(25))
    payoff = {}
    tetas = {}


    profile = [0] * 10
    import tqdm
    def extract_features(features_string):
        return np.array(map(lambda x: float(x.split(':')[1]), features_string.split(' ')[1:6]))
    
    def constract_z(user_features, article_features):
        result_array = []
        for x in xrange(5):
            for y in xrange(5):
                result_array.append(user_features[y] * article_features[x])
        return result_array
        

    try:
        for line in tqdm.tqdm(data):
            splitted = line.split('|')
            user = splitted[1]
            user_features_array = extract_features(splitted[1])
            info = map(int, splitted[0].split(' ')[0:3])
            sum_reward_of_data = sum_reward_of_data + info[2]
            st = time.clock()
            profile[0] += st - time.clock()
            beta = matrix_A0.I * vector_b0.T
            for i in xrange(20):
                st = time.clock()
                article = int(splitted[i + 2].split(' ')[0])
                user_articles[i] = (user, article)

                if article not in article_features:
                    article_features[ article] = extract_features(splitted[i + 2])
                    
                if user_articles[i] not in z:
                    z[user_articles[i]] = np.matrix(constract_z(user_features_array, article_features[article]))
                    

                if article not in matrix_A:
                    matrix_A[article] = np.matrix(np.identity(5))
                    inversed_matrix_A[article] = np.matrix(np.identity(5))
                    vector_b[article] = np.matrix(np.zeros(5))
                    tetas[article] = np.matrix(np.zeros(5)).T
                    matrix_B[article] = np.matrix(np.zeros((25, 5)))



                inverse_matrix = inversed_matrix_A[article] #np.ndarray


                teta = tetas[article]
                z_buf = z[user_articles[i]]
                matrix_B_buf = matrix_B[article]
                user_features = np.matrix(user_features_array)

                #print 2 * z_buf * matrix_A0.I * matrix_B_buf * inverse_matrix * user_features.T 
                s = z_buf * matrix_A0.I * z_buf.T - 2 * z_buf * matrix_A0.I * matrix_B_buf * inverse_matrix * user_features.T + user_features * inverse_matrix * user_features.T + user_features * inverse_matrix * matrix_B_buf.T * matrix_A0.I * matrix_B_buf * inverse_matrix * user_features.T

                payoff_for_user[i] = float(z_buf * beta + user_features * teta + alpha * math.sqrt(s))

            profile[1] += st - time.clock()
            st = time.clock()
            
            winner = np.argwhere(payoff_for_user == np.amax(payoff_for_user))
            chose_article = user_articles[random.choice(map(int, winner))]

            profile[2] += st - time.clock()
            st = time.clock()
            if chose_article == info[1]: #if no information about article it ignors
                real_payoff = info[2] #click information
                sum_reward_of_alg = sum_reward_of_alg + real_payoff
                A = matrix_A[chose_article]
                b = vector_b[chose_article]
                B = matrix_B[chose_article]
                inverse_A = inversed_matrix[chose_article]
                z_buf = z[(user, chose_article)]
                
                matrix_A0 = matrix_A0 + B.T * inverse_A * B
                vector_b0 = vector_b0 + B.T * inverse_A * B

                matrix_A_buf =  A + user_features.T * user_features
                matrix_A[chose_article] = matrix_A_buf
                inverse_A = matrix_A_buf.I
                inversed_matrix_A[chose_article] = inverse_A
                
                matrix_B_buf = B + user_features.T * z_buf
                matrix_B[chose_article] = matrix_B_buf
                
                vector_b[chose_article] = b + real_payoff * user_features
                
                tetas[chose_article] = inversed_matrix_A[chose_article] * (vector_b[chose_article].T - matrix_B[chose_article] * beta)
                
                matrix_A0 = matrix_A0 + z_buf.T * z_buf - B.T * inverse_A * B
                vector_b0 = vector_b0 + real_payoff * z_buf.T  - B.T * inverse_A * B
                
                profile[4] += st - time.clock()
                st = time.clock()
            else:
                #print 'No information about payoff of', chose_article
                pass

            profile[3] += st - time.clock()
            st = time.clock()


        print("--- %s seconds ---" % (time.time() - start_time))
    except KeyboardInterrupt:
        print "Interrupted"
        print profile
    data.close()
    performance.append(sum_reward_of_alg/sum_reward_of_data)
    #average_distance.append(np.sum(distance)/len(distance))

6608it [00:40, 175.57it/s]

In [ ]:
user_features = range(5)
article_features = range(5)
number = 5
result_array = []
result_array = []
for x in xrange(5):
    for y in xrange(5):
        result_array.append(user_features[y]*article_features[x])
print result_array